在沙盒环境下，进行振幅分析

In [1]:
import os, copy, sys
os.environ['ULTRON_DATA'] = 'minden'
os.environ['IGNORE_WARNINGS'] = '0'
import pandas as pd
import numpy as np

In [2]:
from ultron.env import *

/var/log/ultron/2022-10-03.log


In [3]:
from ultron.kdutils.kline import date_week_wave,p_change_stats,wave_change_rate,\
    date_week_win,bcut_change_vc,qcut_change_vc

In [4]:
enable_example_env()

2022-10-03 20:34:32,730 - [env.py:81] - ultron - INFO - enable example env will only read /home/kerry/ultron/rom/sandbox/minden


In [5]:
kl_dict = {}
indicator_data = pd.read_csv(os.path.join(g_project_data, 'indicator.csv'), index_col=0)
indicator_data['trade_date'] = pd.to_datetime(indicator_data['trade_date'])
indicator_data = indicator_data.rename(columns={'preClosePrice':'pre_close','openPrice':'open',
                        'closePrice':'close','highestPrice':'high','lowestPrice':'low',
                        'turnoverVol':'volume'}).drop(['turnoverValue'],axis=1).set_index(['trade_date'])
indicator_data['date'] = indicator_data.index
indicator_data['date'] = indicator_data.index
indicator_data['p_change'] = indicator_data['p_change'] * 100
codes = [300002,300012,300017,300558,300618]
for code in codes:
    dt = indicator_data.reset_index().set_index(
    'code').loc[code].reset_index().set_index('trade_date')
    dt.name = str(code)
    kl_dict[str(code)] = dt

##### 交易日振幅对比分析

In [6]:
date_week_wave(kl_dict)

,300002wave,300012wave,300017wave,300558wave,300618wave
date_week,,,,,
0,4.460901,4.058939,3.895899,4.873177,5.137338
1,4.864159,4.000019,3.731417,4.461239,4.930751
2,4.292335,3.644423,3.592192,4.238644,4.982597
3,4.564277,3.757733,3.677917,4.247642,4.443922
4,4.547521,4.043815,3.646083,4.632381,4.904252


##### 交易日涨跌对比分析

- up_mean:日涨幅平均值
- up_count:上涨走势天数
- down_mean:日跌幅平均值
- up_count:下跌走势天数
- chg_mean:均涨跌比
- chg_ratio:上涨下跌数量比

In [7]:
rval = p_change_stats(kl_dict)
pd.DataFrame(rval)

,code,up_mean,up_count,down_mean,down_count,chg_mean,chg_ratio
0,300002,2.462039,447,-2.553178,420,0.964304,1.064286
1,300012,1.996196,458,-1.801367,427,1.108156,1.072600
2,300017,2.059608,447,-2.201660,433,0.935480,1.032333
3,300558,2.469501,439,-2.236542,459,1.104161,0.956427
4,300618,2.588033,447,-2.595722,452,0.997038,0.988938


##### 振幅统计套利条件分析

In [8]:
## 大于1.8 具备套利条件
rval = wave_change_rate(kl_dict)
pd.DataFrame(rval)

,code,wave_rate
0,300002,1.889106
1,300012,2.091304
2,300017,1.786626
3,300558,1.919680
4,300618,1.890132


##### 交易日涨跌概率分析

In [9]:
date_week_win(kl_dict)

,0,1,300002win,0,1,300012win,0,1,300017win,0,1,300558win,0,1,300618win
date_week,,,,,,,,,,,,,,,
0,87,90,0.508475,91,86,0.485876,82,95,0.536723,91,86,0.485876,78,99,0.559322
1,83,98,0.541436,81,100,0.552486,89,92,0.508287,85,96,0.530387,91,90,0.497238
2,97,86,0.469945,95,88,0.480874,95,88,0.480874,96,87,0.475410,93,90,0.491803
3,101,83,0.451087,88,96,0.521739,96,88,0.478261,105,79,0.429348,97,87,0.472826
4,88,90,0.505618,90,88,0.494382,94,84,0.471910,87,91,0.511236,97,81,0.455056


##### 交易日涨跌区间分析(预定区间)

In [10]:
bcut_change_vc(kl_dict)

,300002,300002rate,300012,300012rate,300017,300017rate,300558,300558rate,300618,300618rate
"(-inf, -10.0]",9,0.009967,0,0.000000,10,0.011074,7,0.007752,8,0.008859
"(-10.0, -7.0]",13,0.014396,1,0.001107,8,0.008859,11,0.012182,21,0.023256
"(-7.0, -3.0]",116,0.128461,82,0.090808,81,0.089701,97,0.107420,111,0.122924
"(-3.0, 0.0]",318,0.352159,362,0.400886,357,0.395349,349,0.386489,316,0.349945
"(0.0, 3.0]",313,0.346622,353,0.390919,343,0.379845,299,0.331118,302,0.334441
"(3.0, 7.0]",105,0.116279,93,0.102990,87,0.096346,119,0.131783,113,0.125138
"(7.0, 10.0]",27,0.029900,11,0.012182,15,0.016611,20,0.022148,28,0.031008
"(10.0, inf]",2,0.002215,1,0.001107,2,0.002215,1,0.001107,4,0.004430


##### 交易日涨跌区间分析(不定区间)

In [11]:
qcut_change_vc(kl_dict)

,300002change,300012change,300017change,300558change,300618change
0,"(-14.443, -3.655]","(-9.18, -2.812]","(-11.045, -3.127]","(-10.538, -3.56]","(-11.548, -4.017]"
1,"(-3.655, -2.303]","(-2.812, -1.722]","(-3.127, -2.008]","(-3.56, -2.224]","(-4.017, -2.528]"
2,"(-2.303, -1.284]","(-1.722, -1.068]","(-2.008, -1.213]","(-2.224, -1.246]","(-2.528, -1.486]"
3,"(-1.284, -0.653]","(-1.068, -0.437]","(-1.213, -0.625]","(-1.246, -0.622]","(-1.486, -0.725]"
4,"(-0.653, 0.0]","(-0.437, 0.0429]","(-0.625, 0.0]","(-0.622, -0.0349]","(-0.725, -0.00966]"
5,"(0.0, 0.585]","(0.0429, 0.541]","(0.0, 0.389]","(-0.0349, 0.457]","(-0.00966, 0.623]"
6,"(0.585, 1.192]","(0.541, 1.103]","(0.389, 1.005]","(0.457, 1.254]","(0.623, 1.245]"
7,"(1.192, 2.229]","(1.103, 2.018]","(1.005, 1.783]","(1.254, 2.377]","(1.245, 2.427]"
8,"(2.229, 3.75]","(2.018, 3.261]","(1.783, 3.272]","(2.377, 4.017]","(2.427, 4.124]"
9,"(3.75, 18.232]","(3.261, 10.98]","(3.272, 18.287]","(4.017, 12.645]","(4.124, 17.716]"
